# Master File Notebook

2/17 -- I created this notebook to apply all the basic data manipulations on the listings and calendar notebook in a separate place so a standard version of the 'Master' dataframe can be loaded from other notebooks rather than repeating the data transformation steps every time

This notebook's purpose is to output the 'master.csv' file in the main directory

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from pandas.plotting import scatter_matrix

In [2]:
os.listdir("Datasources/inside_airbnb") 

['neighbourhoods.geo.json',
 'reviews.csv',
 '.DS_Store',
 'neighbourhoods.geojson',
 'listings_summ.csv',
 'neighbourhoods.csv',
 'stations.geojson',
 'listings.csv',
 'calendar.csv',
 'reviews_summ.csv',
 'neighbourhoods.json']

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
#list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
#neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
#reviews = pd.read_csv('inside_airbnb/reviews.csv')
#reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
listings = listings.loc[listings.room_type == 'Entire home/apt']

In [5]:
cal = calendar.copy()
cal['price'] = cal['price'].astype(str).map(lambda x: x.lstrip('$'))
#Transform Price from Object to Numeric Data Type
cal['price'] = cal['price'].apply(pd.to_numeric, errors='coerce')
cal['price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [6]:
#Just using the calendar listingid, date and price to merge with listings
c = cal.loc[cal.available!='f']
c = c[['listing_id','date','price']]

In [7]:
master = listings.merge(c, how='inner', left_on='id', right_on='listing_id')

In [ ]:
# to csv makes file that is too large even for git lfs
#use to_hdf below
#master.to_csv('master.csv',chunksize=1000)

In [ ]:
"""DataFrame.to_csv(path_or_buf=None, sep=', ', na_rep='', float_format=None, 
                 columns=None, header=True, index=True, index_label=None, mode='w', 
                 encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', 
                 chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, 
                 escapechar=None, decimal='.')"""

In [ ]:
##to_csv is very slow 
#alternatively learn to use hdf5 lib -- conda install pytables
#then write to hdf format: 
#master.to_hdf('master.h5','table',append=True)
#then read in different notebook: pd.read_hdf('store_tl.h5', 'table', where = ['index>2'])

In [13]:
import pyarrow as pa
import pyarrow.parquet as pq

In [15]:
table = pa.Table.from_pandas(master)
pq.write_table(table, 'master.parquet')

In [11]:
#Trying Parquet
master.to_parquet('master.parquet', engine='fastparquet')

RuntimeError: Compression 'snappy' not available.  Options: ['GZIP', 'UNCOMPRESSED']